In [19]:
import pandas as pd
import numpy as np
import sklearn as sk
import csv
from sklearn.linear_model import LinearRegression
#linreg gives straight line
from sklearn.preprocessing import OneHotEncoder
from sklearn.neighbors import KNeighborsRegressor
#doesnt work idk
from sklearn.ensemble import RandomForestClassifier
#doesnt work
from sklearn.ensemble import RandomForestRegressor
#gives me 1 entire value
from sklearn import tree
from sklearn.ensemble import GradientBoostingRegressor
#gives me 1 value
from sklearn.multioutput import MultiOutputRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import MultiTaskLasso
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.linear_model import PassiveAggressiveRegressor
import time
import warnings

In [16]:
train = pd.read_csv("ais_train.csv", sep='|')
test = pd.read_csv("ais_test.csv")
train['time'] = pd.to_datetime(train['time'])
test['time'] = pd.to_datetime(test['time'])

train.tail()

,time,cog,sog,rot,heading,navstat,etaRaw,latitude,longitude,vesselId,portId
1522060,2024-05-07 23:59:07,359.1,13.4,0,1,0,05-08 05:00,52.19131,-5.82223,clh6aqawa0002gh0zypfa5dut,634c4de270937fc01c3a7417
1522061,2024-05-07 23:59:08,12.3,17.1,0,13,0,05-10 03:00,38.96142,-12.00502,61e9f3aeb937134a3c4bfe43,634c4de270937fc01c3a76a1
1522062,2024-05-07 23:59:08,269.8,14.9,-1,270,0,05-15 23:00,49.71372,-5.22042,61e9f43db937134a3c4c0169,634c4de270937fc01c3a787b
1522063,2024-05-07 23:59:08,8.0,18.7,0,6,0,05-08 12:45,38.27895,10.78280,61e9f469b937134a3c4c029b,61d3781293c6feb83e5eb73b
1522064,2024-05-07 23:59:08,336.0,14.3,5,337,0,05-07 23:00,38.98635,-75.13275,62080cff66fc0a8e43c6123a,61d38528b7b7526e1adf3e6f


In [62]:
#endrer tid og 1hoter vessel
warnings.filterwarnings('ignore')

def preprocess(df):
    data = df.copy()
    time3 = []
    for i in data['time']:
        time3.append(int((i.timestamp()-1704067200)/60))
    data['time'] = np.array(time3, dtype = 'float32')
    data = pd.get_dummies(data, columns=['vesselId'], drop_first=True)
    return(data)

test2 = test.copy()
df2 = train.copy()
#ignorerer alle de ubrukelige verdiene 8)
goaldata = df2.drop(columns=['cog','sog','rot','heading','navstat','etaRaw','portId','time','vesselId'])
heading2 = df2.drop(columns=['cog','sog','rot','navstat','etaRaw','portId','time','vesselId','latitude','longitude'])
df2 = preprocess(df2)
trainingdatahead = df2.drop(columns=['cog','sog','rot','navstat','etaRaw','heading','latitude','longitude','portId'])
trainingdata = df2.drop(columns=['cog','sog','rot','navstat','etaRaw','latitude','longitude','portId'])
test2 = preprocess(test2)

#legger til de vesselIdene som ikke er i treningsdataen
for column in trainingdata:
    if column not in test2:
        test2[column] = False
#sorterer
test2 = test2.drop(columns=['ID','scaling_factor','heading'])
test2 = test2.reindex(sorted(test2.columns), axis=1)
trainingdata = trainingdata.reindex(sorted(trainingdata.columns), axis=1)
trainingdatahead = trainingdatahead.reindex(sorted(trainingdatahead.columns), axis=1)
#test3 = test2[test2['vesselId_61e9f469b937134a3c4c029b']== True]
#head
print(trainingdatahead.shape)
print(test2.shape)
test2.head()

(1522065, 688)
(51739, 688)


,time,vesselId_61e9f38eb937134a3c4bfd8d,vesselId_61e9f38eb937134a3c4bfd8f,vesselId_61e9f38eb937134a3c4bfd91,vesselId_61e9f390b937134a3c4bfd93,vesselId_61e9f391b937134a3c4bfd95,vesselId_61e9f391b937134a3c4bfd97,vesselId_61e9f392b937134a3c4bfd99,vesselId_61e9f392b937134a3c4bfd9b,vesselId_61e9f393b937134a3c4bfd9d,...,vesselId_clh682fbq000qghxd3vn5xo01,vesselId_clh682fbq000rghxd2u75zdbn,vesselId_clh6aqawa0000gh0z7nc8e6nd,vesselId_clh6aqawa0001gh0zmijpuho1,vesselId_clh6aqawa0002gh0zypfa5dut,vesselId_clh6aqawa0003gh0zu0aznvt2,vesselId_clh6aqawa0004gh0z12aogec9,vesselId_clh6aqawa0005gh0z64y4xyip,vesselId_clh6aqawa0006gh0zje911dl3,vesselId_clh6aqawa0007gh0z9h6zi9bo
0,184323.0,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,184326.0,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,184330.0,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,184330.0,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,184332.0,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [63]:
predicthead = LinearRegression()
start = time.time()
predicthead.fit(trainingdatahead, heading2.values)
end = time.time()
print(end - start)

38.57087421417236


In [66]:
start = time.time()
res = predicthead.predict(test2)
test4 = test2.copy()
test4['heading'] = res
test4 = test4.reindex(sorted(test4.columns), axis=1)
model = LinearRegression()
model.fit(trainingdata, goaldata)
res = model.predict(test4)
end = time.time()
print(end - start)

76.00320672988892


In [67]:
ramg = [i for i in range(51739)]
res2 = [ramg,[i[1] for i in res],[i[0] for i in res]]
with open('res.csv', 'w') as f:
    write = csv.writer(f)
    write.writerow(["ID","longitude_predicted","latitude_predicted"])
    write.writerows(zip(*res2))
df = pd.read_csv('res.csv')
df.to_csv('res.csv', index=False)

In [14]:
print(res[30:60])

[[ 21.223543   12.142331 ]
 [ 52.386723    8.167897 ]
 [ 42.983345    1.2653694]
 [ 46.05465    -6.9403324]
 [ 57.13925    15.488738 ]
 [ 45.847977  -19.196852 ]
 [ 41.57352     7.148097 ]
 [  5.043148  106.643654 ]
 [ 45.97374    -9.426285 ]
 [ 52.416237    6.42925  ]
 [ 40.388836   -7.309971 ]
 [ 23.673002  -79.89906  ]
 [ 50.8366     -6.16955  ]
 [ 41.54867     2.3367043]
 [  8.223585  107.18321  ]
 [ 46.550602  -99.607    ]
 [ 14.049187    0.8797817]
 [ 33.97502   -67.49801  ]
 [ 45.493145  -28.739084 ]
 [ 36.103863  -21.121782 ]
 [ 16.831139   57.219086 ]
 [ 35.416096   29.116768 ]
 [ 38.36554     5.1916227]
 [ 24.124477  -23.187368 ]
 [ 39.30681   -82.55666  ]
 [ 20.521828   37.541855 ]
 [ 12.190796   28.974121 ]
 [ 46.148582   -6.298256 ]
 [ 47.105938   -2.2182007]
 [ 37.839775   31.279922 ]]
